In [1]:
import os
import json
import pandas as pd
import numpy as np

def extract_from_results_files(root_folder="DRTS_Project-Test-Cases-master"):
    """
    Extract BDR parameters from analysis_results.json files in test case folders.
    """
    # Tables to store the data
    rows = []
    summary_rows = []
    
    # Process each test case folder
    for item in os.listdir(root_folder):
        case_dir = os.path.join(root_folder, item)
        
        # Check if it's a directory and starts with a digit (test case)
        if os.path.isdir(case_dir) and item[0].isdigit():
            result_file = os.path.join(case_dir, "analysis_results.json")
            
            # Check if results file exists
            if os.path.exists(result_file):
                with open(result_file, "r") as f:
                    results = json.load(f)
                
                schedulable = "Yes" if results.get("system_schedulable", False) else "No"
                components = results.get("components", {})
                
                # Extract component parameters
                alphas = []
                deltas = []
                
                for comp_name, comp_info in components.items():
                    alpha = comp_info.get("bdr_alpha", 0)
                    delta = comp_info.get("bdr_delta", 0)
                    
                    rows.append({
                        "Test Case": item,
                        "Component": comp_name,
                        "Alpha (α)": alpha,
                        "Delta (δ)": delta,
                        "Schedulable": schedulable
                    })
                    
                    alphas.append(alpha)
                    deltas.append(delta)
                
                # Create summary entry
                if alphas:  # If we have components
                    critical_alphas = sum(1 for a in alphas if a > 0.9)
                    
                    summary_rows.append({
                        "Test Case": item,
                        "Components": len(components),
                        "Avg Alpha (α)": np.mean(alphas),
                        "Max Alpha (α)": np.max(alphas),
                        "Critical α Count": critical_alphas,
                        "Avg Delta (δ)": np.mean(deltas),
                        "Max Delta (δ)": np.max(deltas),
                        "Schedulable": schedulable
                    })
    
    # Create DataFrames
    component_df = pd.DataFrame(rows)
    summary_df = pd.DataFrame(summary_rows)
    
    # Sort by test case number
    component_df["Case No."] = component_df["Test Case"].apply(lambda x: int(x.split("-")[0]))
    component_df = component_df.sort_values("Case No.").drop("Case No.", axis=1)
    
    summary_df["Case No."] = summary_df["Test Case"].apply(lambda x: int(x.split("-")[0]))
    summary_df = summary_df.sort_values("Case No.").drop("Case No.", axis=1)
    
    # Print tables
    print("Table 1: BDR Parameters by Component")
    print("="*80)
    print(component_df.to_string(index=False))
    
    print("\n\nTable 2: BDR Parameters Summary by Test Case")
    print("="*80)
    print(summary_df.to_string(index=False))
    
    # Save to CSV
    component_df.to_csv("bdr_component_parameters.csv", index=False)
    summary_df.to_csv("bdr_testcase_summary.csv", index=False)
    
    return component_df, summary_df

if __name__ == "__main__":
    extract_from_results_files()

Table 1: BDR Parameters by Component
                 Test Case           Component  Alpha (α)  Delta (δ) Schedulable
          1-tiny-test-case       Camera_Sensor   0.993871        0.0         Yes
         2-small-test-case       Camera_Sensor   0.531613        0.0         Yes
         2-small-test-case     Image_Processor   0.285538       45.0         Yes
        3-medium-test-case       Camera_Sensor   0.497696        0.0         Yes
        3-medium-test-case     Image_Processor   0.287405        0.0         Yes
        3-medium-test-case        Lidar_Sensor   0.227742        7.5         Yes
        3-medium-test-case        Control_Unit   0.622903       12.0         Yes
         4-large-test-case       Camera_Sensor   0.189012        0.0         Yes
         4-large-test-case     Image_Processor   0.195185       15.0         Yes
         4-large-test-case          GPS_Sensor   0.158649        0.0         Yes
         4-large-test-case        Control_Unit   0.618333       15.0    